In [1]:
import pandas as pd
import numpy as np
import os

In [2]:

from autobilis import autobilis_def
from autobonus import autobonus_def
from automobilis import automobilis_def
from autogidas import autogidas_def

In [3]:
# Autobilis
# path_autobilis = os.path.join(r"\csv\autobilis.csv")
path_autobilis = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\EDA\csv\autobilis.csv'
autobilis_df = pd.read_csv(path_autobilis)

In [4]:
# Autobonus
# path_autobonus = os.path.join(r"EDA\csv\autobonus.csv")
path_autobonus = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\EDA\csv\autobonus.csv'
autobonus_df = pd.read_csv(path_autobonus)

In [5]:
# Automobilis
# path_1_automobilis = os.path.join(r"EDA\csv\automobilis.csv")
# path_2_automobilis = os.path.join(r"EDA\csv\automobilis2.csv")
path_1_automobilis = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\EDA\csv\automobilis.csv'
path_2_automobilis = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\EDA\csv\automobilis2.csv'
 
automobilis_df1 = pd.read_csv(path_1_automobilis)
automobilis_df2 = pd.read_csv(path_2_automobilis)

In [6]:
# Autogidas
path_autogidas = r'C:\Users\tvale\OneDrive - Girteka Logistics, UAB\Documents\02_mokymasis\final_poj\EDA\csv\autogidas.csv'
autogidas_df = pd.read_csv(path_autogidas)

In [7]:
# DF 
autobilis = autobilis_def(autobilis_df)
autobonus = autobonus_def(autobonus_df)
automobilis = automobilis_def(automobilis_df1, automobilis_df2)
autogidas = autogidas_def(autogidas_df)

In [8]:
# Apjungiame visus DF 
all_auto = pd.concat([autobilis, autobonus, automobilis, autogidas], ignore_index=True)

In [9]:
# Automobiliu pavadinimu tvarkymas
gamintojas = all_auto.copy()

In [10]:
# Alfa - Romeo
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'Alfa Romeo'), 'Alfa-Romeo', 
    np.where(
    (gamintojas['gamintojas'] == 'AlfaRomeo'), 'Alfa-Romeo', gamintojas['gamintojas']
    )
)

In [11]:
# Land-Rover
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'Land Rover'), 'Land-Rover',
    np.where(
    (gamintojas['gamintojas'] == 'LandRover'), 'Land-Rover', gamintojas['gamintojas']
    )
)

In [12]:
# Mercedes-Benz
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'Mercedes Benz'), 'Mercedes-Benz', gamintojas['gamintojas']
)

In [13]:
# Mini
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'MINI'), 'Mini', gamintojas['gamintojas']
)

In [14]:
# Seat 
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'SEAT'), 'Seat', gamintojas['gamintojas']
)

In [15]:
# SsangYong
gamintojas['gamintojas'] = np.where(
    (gamintojas['gamintojas'] == 'Ssangyong'), 'SsangYong', gamintojas['gamintojas']
)

In [16]:
# Istriname dublikatus 
gamintojas = gamintojas.drop_duplicates()

In [17]:
# Sugrupuojame patikrinti kiek yra kiekvieno galintojo vienetu 
# Jei kiekis mazesnis nei 10 tokius gamintojus istrinti
gamintoju_kiekis = gamintojas.copy()

In [18]:
gamintoju_kiekis = gamintoju_kiekis.groupby(['gamintojas'])[['gamintojas']].count().rename(columns={'gamintojas': 'count'}).reset_index()
gamintoju_kiekis = gamintoju_kiekis.sort_values(by='count')

In [19]:
gamintoju_kiekis = gamintoju_kiekis.loc[gamintoju_kiekis['count'] < 10]

In [20]:
# Prie gamintojas pridedame gamintoju kiekis ir visus kas sutapo istrinti 
gamintojas = pd.merge(
    gamintojas,
    gamintoju_kiekis,
    how='left',
    on='gamintojas'
)

In [21]:
# Istriname tuos gamintojus kuriu kiekis yra mazesnis uz 10 
gamintojas = gamintojas.loc[gamintojas['count'].isna()].reset_index(drop=True)

In [22]:
# Modeliu koregavimas 
modelis = gamintojas.copy()

In [23]:
# Panaikiname tarpus stulpelyje modelis. Didziosios raides
modelis['modelis'] = modelis['modelis'].str.replace(' ', '')
modelis['modelis'] = modelis['modelis'].str.replace(r'\t', '', regex=True)
modelis['modelis'] = modelis['modelis'].str.upper()

In [24]:
# Istriname jei modelis yra -KITA- arba KITAS
modelis = modelis.loc[modelis['modelis'] != '-KITA-']
modelis = modelis.loc[modelis['modelis'] != 'KITAS']

In [25]:
# Metai stulpeli konvertuojame i int reiksme 
modelis['metai'] = modelis['metai'].astype(int)

In [26]:
# Greiciu deze standartizuojame 
modelis['pavaru_deze'] = np.where(
    (modelis['pavaru_deze'] == 'Pusiau automatinė'), 'Automatinė', modelis['pavaru_deze']
)

# Jei nera nurodytas pavaru dezes tipas tokias eilutes eliminuojame 
modelis = modelis.loc[modelis['pavaru_deze'].notna()]

In [27]:
# Jei neturi iraso variklio_turis, pavaru_deze, duru_sk, galia_kw, galia_ag. Jei trinti 0, palikti 1
modelis['remove'] = np.where(
    (
    (modelis['variklio_turis'].isna()) &
    (modelis['pavaru_deze'].isna()) &
    (modelis['duru_sk'].isna()) &
    (modelis['galia_kw'].isna()) &
    (modelis['galia_ag'].isna())
    ), 0, 1
)

modelis = modelis.loc[modelis['remove'] == 1]

In [28]:
# Kuras. Standartizuojame. Jei neturi iraso, tokias eilutes eliminuojame 
modelis['kuras'] = np.where(
    (modelis['kuras'] == 'Benzinas / dujos'), 'Benzinas/Dujos', 
    np.where(
    (modelis['kuras'] == 'Benzinas / elektra'), 'Benzinas/Elektra', 
    np.where(
    (modelis['kuras'] == 'Benzinas/dujos'), 'Benzinas/Dujos', 
    np.where(
    (modelis['kuras'] == 'Dujos'), 'Benzinas/Dujos', modelis['kuras']
    )
    )
    )
    
)

In [29]:
# Jei kuras tulpelyje nera reiksmes arba reiksme lygi Kita tokias eilutes salinti 
modelis = modelis.loc[(modelis['kuras'].notna()) & (modelis['kuras'] != 'Kita')]

In [30]:
# Istriname galia_kw jei nera iraso
modelis = modelis.loc[modelis['galia_kw'].notna()]

In [31]:
# Rida. Jei nan reiksme pakeiciame i 0. Tada konvertuojame i int tipa
modelis['rida'] = modelis['rida'].str.replace('.', '')

modelis['rida'] = np.where(
    (modelis['rida'].isna()), '0', modelis['rida']
)

In [32]:
modelis['rida'] = pd.to_numeric(modelis['rida'])

In [33]:
# Paskaiciuojame ridu vidurki 
ridos_vidurkis = modelis.copy()

In [34]:
# Istriname tas eilutes kur nera irasu 
ridos_vidurkis = ridos_vidurkis.loc[ridos_vidurkis['rida'] != 0]

In [35]:
# Sugrupuojame duomenis ir isvedame vidurki 
ridos_vidurkis = ridos_vidurkis.groupby(['gamintojas', 'modelis', 'metai', 'kuras'])[['rida']].mean().reset_index().rename(columns={'rida': 'rida_mean'})

In [36]:
# Pridedame prie df modelis
car_final = pd.merge(
    modelis,
    ridos_vidurkis,
    how='left',
    on=['gamintojas', 'modelis', 'metai', 'kuras']
)

In [37]:
# Jei rida lygu 0 tokiu atveju graziname reiksme is rida_mean 
car_final['rida_final'] = np.where(
    (car_final['rida'] == 0), car_final['rida_mean'], car_final['rida']
)

In [38]:
# Jei rida_fiinal yra nan reiksme tokias eilutes eliminuoti 
car_final = car_final.loc[car_final['rida_final'].notna()]

In [39]:
# Sugrupuojame paziureti kiek vienetu yra kiekvieno gamintojo 
car_final_gr = car_final.groupby(['gamintojas'])[['gamintojas']].count().rename(columns={'gamintojas': 'count_final'}).reset_index()

In [40]:
# Jei kiekis yra mazesnis uz 10 tokius eliminuojame 
car_final_gr = car_final_gr.loc[car_final_gr['count_final'] > 10]

In [41]:
# Pridedame prie car_final df ir kur nera duomenu istrinti 
car_final = pd.merge(
    car_final,
    car_final_gr,
    how='left',
    on='gamintojas'
)

car_final = car_final.loc[car_final['count_final'].notna()]

In [42]:
# Standartizuojame ridas. Ridu rezis kas 1000 km
car_final['rida_std'] = (car_final['rida_final'] / 1000).apply(np.ceil) * 1000

In [43]:
# Paskaiciuojame amziu. Is einamuju metu atimame 'metai'.
car_final['amzius'] = 2023 - car_final['metai']

In [44]:
# Istriname nereikalingus stulpelius 
car_final = car_final.drop(
    [
        'variklio_turis',
        'duru_sk',
        'count',
        'remove',
        'rida',
        'rida_mean',
        'count_final',
        'galia_ag',
        'metai',
        'rida_final',
        'kebulas'
    ], axis=1
)

In [46]:
# Pervadiname stulpelius 
car_final = car_final.rename(columns={'rida_std': 'rida'})

In [47]:
car_final

,kaina,gamintojas,modelis,pavaru_deze,kuras,galia_kw,rida,amzius
0,6850,Volvo,S60,Mechaninė,Dyzelinas,120,336000.0,13
1,950,Opel,VECTRA,Mechaninė,Benzinas,114,24000.0,17
2,1000,Audi,A6,Automatinė,Benzinas/Dujos,121,271000.0,23
3,2900,BMW,ACTIVEHYBRID7,Automatinė,Benzinas/Elektra,339,185000.0,12
4,1000,Audi,100,Mechaninė,Dyzelinas,85,522000.0,31
...,...,...,...,...,...,...,...,...
12797,4950,Ford,MONDEO,Mechaninė,Dyzelinas,103,334000.0,11
12798,2750,Seat,LEON,Mechaninė,Dyzelinas,125,260000.0,17
12799,4150,Skoda,OCTAVIA,Mechaninė,Dyzelinas,77,274000.0,12
12800,4500,Toyota,RAV4,Mechaninė,Dyzelinas,130,268000.0,16
